To use pytesseract :  
1. It works only on images so convert all pdf to images using `pdf2image`.
2. `Pytesseract` is just a wrapper it need tesseract to be installed on your system by ```brew install tesseract``` for mac.
3. 

In [ ]:
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os

def extract_text_from_scanned_pdf(pdf_path, tesseract_cmd=None):
    """Extract text from a scanned PDF using Tesseract OCR."""
    if tesseract_cmd:
        pytesseract.pytesseract.tesseract_cmd = tesseract_cmd
    # Convert PDF to images
    images = convert_from_path(pdf_path)
    text = ""
    for i, image in enumerate(images):
        page_text = pytesseract.image_to_string(image)
        text += f"\n--- Page {i+1} ---\n{page_text}"
    return text

pdf_file = 'test.pdf'
# If tesseract is not in PATH, specify its location, e.g. '/usr/local/bin/tesseract'
tesseract_path = "/opt/homebrew/bin/tesseract"

extracted_text = extract_text_from_scanned_pdf(pdf_file, tesseract_cmd=tesseract_path)
print(extracted_text)


--- Page 1 ---
GLA University, Mathura
Il-Mid Term Examination, 2011-12
Course: - B.Tech !-Year, |-Sem.
Subject: -Engg.Chemistry Univ. Roll No:-
Time:-90 Minutes Total Marks:-40
Notes:-
1) Answer all questions from Group A (which is) compulsory, Any Two from Group B and Any
Two from Group C.

2) All parts of a question (a, b, etc.) should be answered at one place.

3) Answer should be brief and to-the-point and be supplemented with neat sketches.
4) Any missing or wrong data may be assumed suitably giving proper justification.

5) Figures on the right-hand side margin indicate full marks.

se

Group A
Attempt all the questions 1x16=16
1. Which ion is not there in naturally occurring zeolite.
a) Na’ b) Ca”” c) Al’* d) si*

2. Alkalinity in water can not be due to the presence of

a) HO‘ only b) HO’ and HCO, c) HO and CO;~ d) COs" and HCO;
3. Terylene is a

a) Polyglycol b) Polyester c) Polyamide d) polycarbonate
4, Which one is a co-polymer?

a) PVC b)Buna S c) Teflon d) Nylon-6
5. Ano